<a href="https://colab.research.google.com/github/Usermer/deep-learning-universe/blob/main/02_deep_learning_with_keras_and_tensorfow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#perceptron
import numpy as np
import pandas as pd
from sklearn.linear_model import Perceptron
from sklearn.datasets import load_iris

RANDOM_STATE=42

In [3]:
iris=load_iris(as_frame=True)
iris.data.head()
iris_df=iris.data.copy()
iris_df["target"]=iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
X=iris.data[["petal length (cm)","petal width (cm)"]].values
y=(iris.target==0)#Iris setosa

perceptron_clf=Perceptron(random_state=RANDOM_STATE)
perceptron_clf.fit(X,y)


X_new=[[1.5,0.6]]
y_pred=perceptron_clf.predict(X_new)
y_pred

array([ True])

**Regression MLPs**

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

housing=fetch_california_housing()
X=housing.data
y=housing.target
X_train_full,X_test,y_train_full,y_test=train_test_split(X,y,test_size=0.2,random_state=RANDOM_STATE)
X_train,X_val,y_train,y_val=train_test_split(X_train_full,y_train_full,test_size=0.2,random_state=RANDOM_STATE)

mlp_reg=MLPRegressor(hidden_layer_sizes=[50,50,50],random_state=RANDOM_STATE)
pipeline=make_pipeline(StandardScaler(),mlp_reg)
pipeline.fit(X_train,y_train)
y_pred=pipeline.predict(X_val)
rmse=np.sqrt(mean_squared_error(y_val,y_pred))
rmse

np.float64(0.5501144637946774)

**Implementing MLPs with Keras**

***building an Image Classifiert Using the Sequential API***

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
from tensorflow.keras import Sequential

RANDOM_STATE=42
tf.keras.utils.set_random_seed(RANDOM_STATE)

Training a neural network involves randomness in many places:

*  Weight initialization



*  Shuffling training data

*  Dropout

*  Some GPU operations

In [7]:
fashion_mnist=datasets.fashion_mnist.load_data()


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
(X_train_full,X_test),(y_train_full,y_test)=fashion_mnist
X_train,y_train=X_train_full[:50000],y_train_full[:50000]
X_val,y_val=X_train_full[50000:],y_train_full[50000:]

In [9]:
X_train.shape


(50000, 28, 28)

When loading MNIST on Fashion MNIST using Keras rather than Scikit-learn , one important difference is that every image is represented as a 28 x 28 array rather than 1D array of size:784

In [10]:
#X_train[1]
y_train[1]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  13,  67,   0,
          0,   0,   0,  50,  38,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   8, 120, 209, 226, 247, 237,
        255, 255, 255, 247, 238, 235, 172,  72,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0, 137, 239, 252, 243, 234, 229, 238,
        244, 246, 240, 230, 232, 239, 248, 251, 194,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 102, 255, 231, 228, 227, 228, 233, 230,
        230, 229, 228, 232, 232, 231, 227, 224, 252, 179,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 233, 241, 229, 231, 255, 255, 238, 231,
        227, 238, 246, 228, 230, 227, 234, 235, 229, 241,  20,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 248, 241, 231, 255, 149,  47, 252, 228,
        255, 242, 216, 238, 232, 255, 228, 220, 234, 250,  54,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 255, 240, 232, 255,  15,   0, 255, 237,
        191,   0,   0, 214, 255,  13, 123, 255, 234, 252, 114,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   6, 255, 238, 239, 255, 177,   0, 255, 255,
          0, 130, 116,  47,  65,  43,  37, 255, 236, 249, 162,   0,   0,
          0,   0],
       [  0,   0,   0,   0,  32, 255, 236, 245, 255, 204,   0, 255,  84,
          0,  37,  28,  31,   0,  25,  13, 255, 236, 249, 199,   0,   0,
          0,   0],
       [  0,   0,   0,   0,  53, 255, 236, 250, 250, 231,   2, 255,  21,
          0, 221, 255, 236,  54, 245, 198, 243, 238, 245, 223,   0,   0,
          0,   0],
       [  0,   0,   0,   0,  80, 255, 237, 250, 240, 255,   0,   0,  39,
        157,   0,   0, 215,  94,  20, 126, 255, 237, 239, 250,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 101, 255, 235, 253, 244, 243, 133, 138, 208,
        255, 201, 214, 255, 230,   7, 174, 255, 240, 238, 255,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 126, 255, 233, 255, 248, 233, 255, 255, 240,
        232, 243, 243, 231, 251, 255, 255, 254, 243, 238, 255,   3,   0,
          0,   0],
       [  0,   0,   0,   0, 147, 255, 233, 249, 181, 243, 227, 224, 230,
        234, 230, 230, 235, 228, 235, 222, 207, 255, 236, 255,  35,   0,
          0,   0],
       [  0,   0,   0,   0, 163, 255, 245, 221,  86, 255, 233, 233, 235,
        236, 234, 234, 234, 232, 242, 231, 125, 255, 236, 255,  55,   0,
          0,   0],
       [  0,   0,   0,   0, 181, 254, 255, 200,  69, 255, 228, 232, 234,
        235, 234, 234, 233, 235, 241, 237,  70, 255, 235, 246,  57,   0,
          0,   0],
       [  0,   0,   0,   0, 197, 247, 255, 188, 110, 255, 224, 233, 234,
        234, 234, 234, 234, 234, 240, 253,  69, 255, 236, 248,  77,   0,
          0,   0],
       [  0,   0,   0,   0, 200, 246, 255, 149, 145, 255, 223, 235, 234,
        235, 235, 235, 234, 237, 233, 255,  47, 255, 239, 249,  98,   0,
          0,   0],
       [  0,   0,   0,   0, 204, 243, 255, 111, 173, 255, 227, 235, 235,
        236, 235, 235, 235, 239, 229, 255,  19, 227, 246, 249, 110,   0,
          0,   0],
       [  0,   0,   0,   0, 196, 240, 255, 109, 213, 250, 229, 235, 235,
        236, 235, 237, 236, 237, 226, 255,  55, 203, 251, 245, 120,   0,
          0,   0],
       [  0,   0,   0,   0, 192, 243, 255, 114, 232, 240, 232, 235, 235,
        236, 234, 237, 236, 235, 229, 255, 134, 171, 252, 244, 137,   0,
          0,   0],
       [  0,   0,   0,   0, 189, 251, 255, 154, 238, 233, 236, 234, 235,
        236, 235, 238, 236, 235, 232, 255, 166, 125, 255, 243, 142,   0,
          0,   0],
       [  0,   0,   0,   0, 183, 252, 255, 171, 247, 232, 234, 234, 233,
        233, 232, 234, 233, 234, 233, 240, 223, 128, 255, 242, 151,   0,
          0,   0],
       [  0,   0,   0,   0, 178, 243, 255,  57, 238, 241, 238, 238, 238,
        237, 236, 237, 237, 240, 237, 254, 176,  52, 255, 239, 157,   0,
          0,   0],
       [  0,   0,   0,   0, 188, 240, 25

For simplicity, we'll scale the pixel intensities from 0-1 range rather that 0-255 by dividing thel by 255

In [11]:
X_train,X_val,X_test=X_train/255,X_val/255,X_test/255

For fashion-mnist we need a list of class names to know what we are dealing with:

In [12]:
class_names=["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

**Creatig the model using the `sequential` API**

**Sequential API** :is the simplest way to build neural networks when layers are stacked one after another.

In [17]:
model=Sequential([
    keras.layers.Input(shape=[28,28],name="input"),
    keras.layers.Flatten(),
    keras.layers.Dense(300,activation='relu',name="dense_1"),
    keras.layers.Dense(100,activation="relu",name="dense_2"),
    keras.layers.Dense(10,activation="softmax",name="output")
])



In [18]:
#the layers are accessible via layers attribute
model.layers

[<Flatten name=flatten_1, built=True>,
 <Dense name=dense_1, built=True>,
 <Dense name=dense_2, built=True>,
 <Dense name=output, built=True>]

**A Sequential model behaves very much like an array of layers**

In [20]:
#model.pop()
#model.add(keras.layers.Dense(10,activation="softmax"))
len(model.layers) #flatten+3 dense layers

4

In [21]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 266,610 (1.02 MB)

 Trainable params: 266,610 (1.02 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
dense_1=model.layers[1]
weights,bias=dense_1.get_weights()
weights #random in the start

array([[ 0.04701172,  0.02030142,  0.00017369, ..., -0.05407985,
        -0.05218757, -0.03243225],
       [ 0.05967203,  0.00751173, -0.00814871, ...,  0.0057645 ,
        -0.01201727, -0.03320028],
       [ 0.06289387, -0.01109221, -0.00166835, ..., -0.06984416,
         0.01197089,  0.01446974],
       ...,
       [ 0.02109563,  0.00076367,  0.05764256, ..., -0.0706034 ,
         0.05628157, -0.01037535],
       [ 0.03224595,  0.04372283, -0.03925233, ...,  0.04851261,
         0.01097853,  0.02257331],
       [ 0.02585019,  0.00615785,  0.05370931, ...,  0.06699671,
        -0.03691747,  0.07247207]], dtype=float32)

🔹 Why are the weights random at the start?

**To break symmetry:**

If all weights started the same:

*  All neurons would learn the same thing

*  The network would be useless

**Random initialization ensures:**
Each neuron learns different features

In [32]:
bias#0

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

**Compiling the model**